## Downlowding a Dataset from Kaggle

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d zackyboi/puregym-customer-reviews-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/zackyboi/puregym-customer-reviews-sentiment-analysis
License(s): unknown
 60% 5.00M/8.31M [00:00<00:00, 52.3MB/s]
100% 8.31M/8.31M [00:00<00:00, 76.9MB/s]


In [5]:
import zipfile

with zipfile.ZipFile('/content/puregym-customer-reviews-sentiment-analysis.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

## Dataset

In [6]:
import pandas as pd

df = pd.read_csv('/content/PureGym Customer Reviews.csv')

In [7]:
df

,Total Reviews,Profile Image,Customer Name,Country Code,Rating,Date Experienced,Review Title,Review,Owner,Owner Date Posted,Owners Response
0,1,EC,Elena Curtis,GB,4,2023-12-17,"A well equipped gym, lacked cleaning facilities",a well equipped gym and only noticed one machi...,PureGym UK,No Date Provided,No Response Provided
1,4,Custom Image,Graham F,GB,4,2023-12-13,Overall a good experience,overall a good experience. its still too cramp...,PureGym UK,2 days ago,Hi Graham. Thanks for taking the time to leave...
2,8,IO,Ian Oldham,GB,5,2023-12-12,Excellent gym,"excellent gym, friendly environment, plenty of...",PureGym UK,2 days ago,Hi Ian. Thanks for taking the time to leave th...
3,15,MB,Mick Bayley,GB,5,2023-11-12,I'm going to be so fit,havnt been a gym member for 20 years but this ...,PureGym UK,No Date Provided,No Response Provided
4,1,LU,Lucy,GB,5,2023-12-13,Chloe Lawrence PT at PureGym,i have been training with chloe lawrence at pu...,PureGym UK,2 days ago,Hi Lucy. Thanks for taking the time to leave t...
...,...,...,...,...,...,...,...,...,...,...,...
31644,12,OL,Olga,GB,4,2015-10-23,Cheap and cheerful,if you are not very picky and dont expect saun...,PureGym UK,No Date Provided,No Response Provided
31645,12,Custom Image,Paul John Cox,GB,5,2015-02-18,NO excuses now!,i admit im not in the best shape and intend to...,PureGym UK,No Date Provided,No Response Provided
31646,22,Custom Image,Charlotte,DK,5,2015-02-07,Great classes,"the classes are great, i signed up due to the ...",PureGym UK,No Date Provided,No Response Provided
31647,15,RA,Ross Anderson,GB,4,2014-08-12,Decent gym,the pure gyms are well equipped and have every...,PureGym UK,No Date Provided,No Response Provided


In [12]:
df['Review Title'].iloc[1], df['Review'].iloc[1], df['Rating'].iloc[1], df['Owners Response'].iloc[1]

('Overall a good experience',
 'overall a good experience. its still too cramped compared with the old redditch gym, but numbers seem to be reducing a little so equipment is a little more readily available. the changing rooms are far too small though (take a look at sunderland pure gym for comparison). one jewel in the crown is mark fisher. this guys classes are "insane" (in the nicest possible way!). book his classes early to avoid disappointment.',
 4,
 'Hi Graham. Thanks for taking the time to leave this review, we are happy to see this is 4 stars.\n\nWe want to provide our members with a safe space in which they can get a great workout in and do so, in a clean and friendly environment.\n\nIt’s great to hear you enjoy the classes and using the gym overall!\n\nThanks, PureGym')

In [13]:
id=31648
df['Review Title'].iloc[id], df['Review'].iloc[id], df['Rating'].iloc[id], df['Owners Response'].iloc[id]

('poor customer service',
 'called to try and change my direct debit date over only to be met by someone who clearly couldnt be bothered to help, the call was disconnected so i called back to advise if i was unable to carry out this change i wanted to cancel my membership... 2nd person i spoke with again had the same attitude and jus told me to cancel the direct debit... good customer service is something there dept who deal with direct debits clearly lack!',
 1,
 'No Response Provided')

In [14]:
id=31645
df['Review Title'].iloc[id], df['Review'].iloc[id], df['Rating'].iloc[id], df['Owners Response'].iloc[id]

('NO excuses now!',
 'i admit im not in the best shape and intend to improve my fitness as much as i can, i used to be a member of the local warwick county council gym but sadly found their prices a little steep! as i work mostly during the night i was restricted in when i could actually get some decent time in the gym! not good! so i wanted to find somewhere not only cheaper but would also allow me flexibility in when i could get in on my terms! puregym flagged up  as the best candidate and allows me all that i wanted in getting in better shape, the big bonus for me was the amazing amount of kit to explore and not having to wait for a chance to use it! plenty for all! price you pay never goes up so if you just want to get some gym time in for a reseanable price i seriously recommend you go to your nearest puregym! trust me they are everywhere!',
 5,
 'No Response Provided')